# Exploring the Planck Likelihood Code - plc / Clik
Author: João Victor S. Rebouças, December 2021

This is a notebook for learning how to use clik independently.

Follow the installation guide at https://github.com/CoupleDE-UNESP/Joao_repository/blob/main/My_Notes/installing_planck_likelihood.md

A full description of the likelihood is in https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/CMB_spectrum_%26_Likelihood_Code

In [5]:
import sys
import numpy as np
import matplotlib.pyplot as plt
# We need to point to the system the path where clik is installed
sys.path.append("/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/.local/lib/python/site-packages")
import clik
import camb

In [6]:
# Paths for clik files
# Clik files are directories ending with .clik
# Clik files determine the likelihood and provide the Pĺanck data
# Data can be downloaded via Cobaya - cobaya-install cosmo -p /desired/path/
# Alternatively, you can download the data from https://pla.esac.esa.int/pla/aio/product-action?COSMOLOGY.FILE_ID=COM_Likelihood_Data-baseline_R3.00.tar.gz
path_to_data =  '/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/external_modules/data/planck/plc_3.0'
highlpath    =  path_to_data+'/hi_l/plik/plik_rd12_HM_v22b_TTTEEE.clik/'
lowlTTpath   =  path_to_data+'/low_l/commander/commander_dx12_v3_2_29.clik/'
lowlEEpath   =  path_to_data+'/low_l/simall/simall_100x143_offlike5_EE_Aplanck_B.clik/'
lensingpath  =  path_to_data+'/lensing/smicadx12_Dec5_ftl_mv2_ndclpp_p_teb_consext8.clik_lensing/'

In [7]:
# Initializing the likelihoods
highl_like   = clik.clik(highlpath)
lowl_TT_like = clik.clik(lowlTTpath)
lowl_EE_like = clik.clik(lowlEEpath)
lensing_like = clik.clik_lensing(lensingpath)

----
clik version dba7f161d364
  smica
Checking likelihood '/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/external_modules/data/planck/plc_3.0/hi_l/plik/plik_rd12_HM_v22b_TTTEEE.clik/' on test data. got -1172.47 expected -1172.47 (diff -4.34054e-07)
----
----
clik version dba7f161d364
  gibbs_gauss b13c8fda-1837-41b5-ae2d-78d6b723fcf1
Checking likelihood '/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/external_modules/data/planck/plc_3.0/low_l/commander/commander_dx12_v3_2_29.clik/' on test data. got -11.6257 expected -11.6257 (diff -1.07424e-09)
----
Initializing SimAll
----
clik version dba7f161d364
  simall simall_EE_BB_TE
Checking likelihood '/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/external_modules/data/planck/plc_3.0/low_l/simall/simall_100x143_offlike5_EE_Aplanck_B.clik/' on test data. got -197.99 expected -197.99 (diff -4.1778e-08)
----
Checking lensing likelihood '/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/external_modules/data/planck/plc_3.0/lensing/smicadx12_Dec5_ftl_

In [14]:
# Generating Cls with CAMB
lcdmcosmology = camb.set_params(
                                ombh2     = 0.02238,
                                omch2     = 0.120175,
                                thetastar = 0.010409,
                                tau       = 0.054308,
                                As        = 2.1005*10**(-9),
                                ns        = 0.9675,
                                max_l     = 2508
                               )
lcdmresults   = camb.get_results(lcdmcosmology)
lcdmdls       = lcdmresults.get_lensed_scalar_cls(CMB_unit='muK')
lcdmdltt, lcdmdlee, lcdmdlte = lcdmdls[:,0], lcdmdls[:,1], lcdmdls[:,3]
ells   = np.arange(2508+1)
factor = ells*(ells + 1)/(2*np.pi)
cls = {"TT": lcdmdltt/factor,
       "TE": lcdmdlte/factor,
       "EE": lcdmdlee/factor}

/home/joaov/anaconda3/envs/cocoa/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  
/home/joaov/anaconda3/envs/cocoa/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
/home/joaov/anaconda3/envs/cocoa/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


## First Example: Calculating $\chi^2$ for low-l TT spectrum
Clik objects are callable. The expected argument is an array with the corresponding $C_\ell$s and nuisance parameters.

For instance, let's check the low-l TT likelihood. To check what it needs, use the methods `get_lmax()` and `get_extra_parameter_names()`

In [15]:
print('Max ells for lowl_TT_like:', lowl_TT_like.get_lmax())
print('Apart from Cls, need to insert:', lowl_TT_like.get_extra_parameter_names())

Max ells for lowl_TT_like: (29, -1, -1, -1, -1, -1)
Apart from Cls, need to insert: ('A_planck',)


This means that, to calculate the low-l TT likelihood, you need to insert $C_\ell^{TT}$ from $\ell = 0$ to $\ell = 29$ and the nuisance parameter `A_planck`, the Planck absolute calibration.

These must be passed to the clik object in an 1D array. The order of the array must always be:
 - $C_\ell^{TT}$ from $\ell = 0$ to $\ell = \ell_{max}$
 - $C_\ell^{EE}$ from $\ell = 0$ to $\ell = \ell_{max}$
 - $C_\ell^{BB}$ from $\ell = 0$ to $\ell = \ell_{max}$
 - $C_\ell^{TE}$ from $\ell = 0$ to $\ell = \ell_{max}$
 - $C_\ell^{TB}$ from $\ell = 0$ to $\ell = \ell_{max}$
 - $C_\ell^{EB}$ from $\ell = 0$ to $\ell = \ell_{max}$
 - Nuisance parameters in the order indicated by `get_extra_parameter_names()`

In [16]:
lowl_TT_lmax = lowl_TT_like.get_lmax()[0]
lowl_TT_cls  = cls['TT'][0:lowl_TT_lmax + 1]
aplanck = 1 # Example
cls_and_nuisance = np.append(lowl_TT_cls,aplanck)
example_loglike = lowl_TT_like(cls_and_nuisance)
print('TT low-l log-likelihood for my example:', example_loglike)

TT low-l log-likelihood for my example: [-11.43890684]


## Second Example: high-l TTTEEE
There are many nuisance parameters but we can get them from a Cobaya sample. For the calculation, we only need to follow the previous steps.

In [28]:
print('Max ells for high-l TTTEEE likelihood:', highl_like.get_lmax())
print('Apart from Cls, need to insert:', highl_like.get_extra_parameter_names())
needed_nuisance_params_TTTEEE = highl_like.get_extra_parameter_names()

Max ells for high-l TTTEEE likelihood: (2508, 2508, -1, 2508, -1, -1)
Apart from Cls, need to insert: ('A_cib_217', 'cib_index', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_EE_A_100', 'galf_EE_A_100_143', 'galf_EE_A_100_217', 'galf_EE_A_143', 'galf_EE_A_143_217', 'galf_EE_A_217', 'galf_EE_index', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 'galf_TE_A_217', 'galf_TE_index', 'A_cnoise_e2e_100_100_EE', 'A_cnoise_e2e_143_143_EE', 'A_cnoise_e2e_217_217_EE', 'A_sbpx_100_100_TT', 'A_sbpx_143_143_TT', 'A_sbpx_143_217_TT', 'A_sbpx_217_217_TT', 'A_sbpx_100_100_EE', 'A_sbpx_100_143_EE', 'A_sbpx_100_217_EE', 'A_sbpx_143_143_EE', 'A_sbpx_143_217_EE', 'A_sbpx_217_217_EE', 'calib_100T', 'calib_217T', 'calib_100P', 'calib_143P', 'calib_217P', 'A_pol', 'A_planck')


In [53]:
# I want to get some default values for the Planck nuisance parameters
# One place to get them is in Cobaya! Check for instance `cobaya/likelihoods/planck_2018_highl_plik/params_TT.yaml`
# Yaml files can be loaded as dictionaries in Python using the pyyaml library
# Each entry is a nuisance parameter!
import yaml
planck_yaml_files =  [
                     'calib_pol',
                     'calib_temp',
                     'calib',
                     'EE',
                     'TE',
                     'TT'                    
                    ]
yamls = {} # Each yaml is a dictionary whose keys are the nuisance parameters
for name in planck_yaml_files:
    with open(f'/home/joaov/cosmo/cocoa_all/open_cocoa2/Cocoa/cobaya/cobaya/likelihoods/planck_2018_highl_plik/params_{name}.yaml', 'r') as file:
        yamls = {**yamls, **yaml.safe_load(file)}

In [ ]:
# Checking if all parameters are in here
list(yamls.keys()).sort() == list(needed_nuisance_params_TTTEEE).sort()

In [83]:
standard_nuisance_params_TTTEEE = np.zeros(47) # Default values for the TTTEEE nuisance parameters, in order
for i, nuisance_param in enumerate(needed_nuisance_params_TTTEEE):
    try:
        standard_nuisance_params_TTTEEE[i] = yamls[nuisance_param]
    except TypeError:
        try:
            standard_nuisance_params_TTTEEE[i] = yamls[nuisance_param]['value']
        except KeyError:
            standard_nuisance_params_TTTEEE[i] = yamls[nuisance_param]['ref']['loc']

In [84]:
standard_nuisance_params_TTTEEE

array([ 6.7000e+01, -1.3000e+00,  0.0000e+00,  7.0000e+00,  2.5700e+02,
        4.7000e+01,  4.0000e+01,  1.0400e+02,  0.0000e+00,  7.0000e+00,
        9.0000e+00,  2.1000e+01,  8.0000e+01,  5.5000e-02,  4.0000e-02,
        9.4000e-02,  8.6000e-02,  2.1000e-01,  7.0000e-01, -2.4000e+00,
        1.3000e-01,  1.3000e-01,  4.6000e-01,  2.0700e-01,  6.9000e-01,
        1.9380e+00, -2.4000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
        1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
        1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
        1.0002e+00,  9.9805e-01,  1.0210e+00,  9.6600e-01,  1.0400e+00,
        1.0000e+00,  1.0000e+00])

In [85]:
# Formatting the input
needed_cls = np.append(cls['TT'], cls['EE'])
needed_cls = np.append(needed_cls, cls['TE'])
input_to_like_TTTEEE = needed_cls
for element in standard_nuisance_params_TTTEEE:
    input_to_like_TTTEEE = np.append(input_to_like_TTTEEE, float(element))

In [86]:
# Finally, calculating high-l TTTEEE likelihood
example2_loglike = highl_like(input_to_like_TTTEEE)
print('TTTEEE high-l log-likelihood for my example:', example2_loglike)

TTTEEE high-l log-likelihood for my example: [-1537.00959506]


## Third Example: Lensing
Lensing works a bit differently: now we need to insert $C_\ell^{\phi\phi}$, $C_\ell^{TT}$, $C_\ell^{EE}$, $C_\ell^{TE}$ in this order, up until $\ell = 2500$. You can generate a fiducial lensing power spectrum $C_\ell^{\phi\phi}$ with the method `get_clpp_fid()`

In [11]:
# Checking
print(lensing_like.get_lmax())
print(lensing_like.get_extra_parameter_names())

[2500 2500 2500   -1 2500   -1   -1]
('A_planck',)


In [20]:
clpp = lensing_like.get_clpp_fid()
aplanck = 1
input_to_like_lens = clpp
for cl in [cls['TT'], cls['EE'], cls['TE']]:
    input_to_like_lens = np.append(input_to_like_lens, cl[:2501])
input_to_like_lens = np.append(input_to_like_lens, [aplanck])

In [21]:
example3_loglike = lensing_like(input_to_like_lens)
print('Lensing log-likelihood for my example:', example3_loglike)

Lensing log-likelihood for my example: [-25.28993146]


## Final Example: All likelihoods
Now, I'm going to use high-l TTTEEE, low-l TT+EE and lensing (smica): these likelihoods form the Planck baseline likelihood

In [112]:
print(lensing_like.get_lmax())
print('low-l TT needs: Cl^TT up until l =', lowl_TT_like.get_lmax()[0], 'and', lowl_TT_like.get_extra_parameter_names(), '\n')
print('low-l EE needs: Cl^EE up until l =', lowl_EE_like.get_lmax()[1], 'and', lowl_EE_like.get_extra_parameter_names(), '\n')
print('high-l TTTEEE needs: Cl^TT, Cl^EE and Cl^TE up until l =', highl_like.get_lmax()[0], 'and', highl_like.get_extra_parameter_names(), '\n')
print('Lensing needs: Cl^\phi\phi, Cl^TT, Cl^EE and Cl^TE up until l =', lensing_like.get_lmax()[0], 'and', lensing_like.get_extra_parameter_names())

[2500 2500 2500   -1 2500   -1   -1]
low-l TT needs: Cl^TT up until l = 29 and ('A_planck',) 

low-l EE needs: Cl^EE up until l = 29 and ('A_planck',) 

high-l TTTEEE needs: Cl^TT, Cl^EE and Cl^TE up until l = 2508 and ('A_cib_217', 'cib_index', 'xi_sz_cib', 'A_sz', 'ps_A_100_100', 'ps_A_143_143', 'ps_A_143_217', 'ps_A_217_217', 'ksz_norm', 'gal545_A_100', 'gal545_A_143', 'gal545_A_143_217', 'gal545_A_217', 'galf_EE_A_100', 'galf_EE_A_100_143', 'galf_EE_A_100_217', 'galf_EE_A_143', 'galf_EE_A_143_217', 'galf_EE_A_217', 'galf_EE_index', 'galf_TE_A_100', 'galf_TE_A_100_143', 'galf_TE_A_100_217', 'galf_TE_A_143', 'galf_TE_A_143_217', 'galf_TE_A_217', 'galf_TE_index', 'A_cnoise_e2e_100_100_EE', 'A_cnoise_e2e_143_143_EE', 'A_cnoise_e2e_217_217_EE', 'A_sbpx_100_100_TT', 'A_sbpx_143_143_TT', 'A_sbpx_143_217_TT', 'A_sbpx_217_217_TT', 'A_sbpx_100_100_EE', 'A_sbpx_100_143_EE', 'A_sbpx_100_217_EE', 'A_sbpx_143_143_EE', 'A_sbpx_143_217_EE', 'A_sbpx_217_217_EE', 'calib_100T', 'calib_217T', 'calib_1